In [1]:
import pandas as pd
import polars as pl
import numpy as np
import scipy

import category_encoders as ce
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

from typing import Tuple
import datetime

import warnings
warnings.filterwarnings('ignore')

RANDOM_STATE = 42

In [2]:
data = pl.read_csv('../data/data.csv', try_parse_dates=True).sort('date_time')
data = data.drop('campaign_clicks')
data = (
    # по условию нужно убрать такие случаи, что banner_id != banner_id0
    data.filter(pl.col('banner_id') == pl.col('banner_id0'))
    # также отфильтруем g1 = None
    .filter(pl.col('g1').is_not_null())
    # banner_id теперь нам не нужен
    .drop('banner_id')
)
data.head()

date_time,zone_id,oaid_hash,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,impressions,clicks
datetime[μs],i64,i64,i64,i64,i64,f64,f64,f64,i64,f64,f64,f64,i64,i64
2021-09-01 00:02:49,30,5236744527665721365,0,7,596,0.05,0.06456,-4.312062,603,0.05,0.05651,-4.370191,1,0
2021-09-26 00:00:00,1,7416450538971744701,2,15,188,0.008,0.014186,-3.811444,11754925,0.33,0.012351,-2.106896,1,0
2021-09-26 00:00:00,17,4618896985986523547,1,1,3,0.012,0.012615,-3.844725,99,0.006,0.054492,-3.950419,1,0
2021-09-26 00:00:00,86,7650534412494426894,0,1,61,0.005,0.077753,-3.943275,124,0.003,0.063488,-3.846605,1,0
2021-09-26 00:00:00,19,1861020308801330987,1,0,36,0.054,0.053578,-5.048463,12137469,9.5,0.0115,-4.525252,1,0


Закодируем время в виде синуса/косинуса от часа дня и дня недели

In [3]:
data = (
    data
    .with_columns([
        pl.col('date_time').apply(lambda x: x.hour).alias('hour'),
        pl.col('date_time').apply(lambda x: x.weekday()).alias('weekday'),
    ])
    .with_columns([
        pl.col('hour').apply(lambda x: np.sin(2 * np.pi * x / 24)).alias('sin_hour'),
        pl.col('hour').apply(lambda x: np.cos(2 * np.pi * x / 24)).alias('cos_hour'),
        pl.col('weekday').apply(lambda x: np.sin(2 * np.pi * x / 7)).alias('sin_weekday'),
        pl.col('weekday').apply(lambda x: np.cos(2 * np.pi * x / 7)).alias('cos_weekday'),
    ])
    .drop('hour', 'weekday')
)

Для тестирования решения будем использовать последний день, а для валидации – предпоследний

In [4]:
test_date_threshold = data['date_time'].max().replace(hour=0, minute=0, second=0, microsecond=0)
print(f'test date threshold: {test_date_threshold}')

test date threshold: 2021-10-02 00:00:00


In [5]:
train_data = data.filter(pl.col('date_time') < test_date_threshold)
test_data = data.filter(pl.col('date_time') >= test_date_threshold)
print(f'строчек в тренировочной выборке: {len(train_data)}')
print(f'строчек в тестовой выборке: {len(test_data)}')

# sanity check
assert len(train_data) + len(test_data) == len(data)

строчек в тренировочной выборке: 12056598
строчек в тестовой выборке: 1890562


In [6]:
def transform_data(data: pl.DataFrame, mode: int = 0) -> Tuple[pd.DataFrame, pd.Series]:
    return (
        data
        # переименуем колонки для выбранного баннера
        .rename({f'{col}{mode}': col for col in ['rate', 'banner_id', 'g', 'coeff_sum']})
        # уберем колонки для другого баннера
        .drop([f'{col}{1 - mode}' for col in ['rate', 'banner_id', 'g', 'coeff_sum']])
    )

train_data = transform_data(train_data)
test_data_0 = transform_data(test_data, mode=0)
test_data_1 = transform_data(test_data, mode=1)

In [7]:
assert train_data.columns == test_data_0.columns and train_data.columns == test_data_1.columns

Чтобы закодировать категориальные признаки, воспользуемся target encoding для признаков, у которых много уникальных значений и One-hot encoding для признаков, у которых немного уникальных значений, чтобы не раздувать память. Для этого я использовал библиотеку `category-encoders`

In [8]:
target_col = 'clicks'
drop_columns = ['date_time', 'impressions', 'clicks']

loo_encoder = ce.leave_one_out.LeaveOneOutEncoder(cols=['zone_id', 'banner_id', 'oaid_hash'])
loo_encoder.fit(train_data.drop(drop_columns).to_pandas(), train_data[target_col].to_pandas())

ce_one_hot_encoder = ce.OneHotEncoder(cols=['os_id', 'country_id'])
ce_one_hot_encoder.fit(train_data.drop(drop_columns).to_pandas(), train_data[target_col].to_pandas())

OneHotEncoder(cols=['os_id', 'country_id'])

In [10]:
def feature_engineering(data: pl.DataFrame) -> Tuple[pd.DataFrame, pd.Series]:
    X, y = data.drop(drop_columns).to_pandas(), data[target_col].to_pandas()
    X = loo_encoder.transform(X, y)
    X = ce_one_hot_encoder.transform(X, y)
    return X, y

train_X, train_y = feature_engineering(train_data)
test_X_0, test_y_0 = feature_engineering(test_data_0)
test_X_1, _ = feature_engineering(test_data_1)

In [11]:
def create_model(**kwargs):
    return LogisticRegression(**kwargs)

In [12]:
def eval_model(model, X, y):
    y_pred = model.predict_proba(X)[:, 1]
    print(f'ROC AUC = {roc_auc_score(y, y_pred)}')
    print(f'Log loss = {log_loss(y, y_pred)}')

In [13]:
model = create_model(random_state=RANDOM_STATE, max_iter=50, C=1.)
model.fit(train_X, train_y)

LogisticRegression(max_iter=50, random_state=42)

In [50]:
eval_model(model, test_X_0, test_y_0)

ROC AUC = 0.7547338783043653
Log loss = 0.142684959688648


$\pi_0$

> На оснований этих данных вы можете посчитать $\pi_0$, для этого вам достаточно только в каждом событии определить вероятность того, что одна нормальная величина больше другой.

Для этого воспользуемся свойством, что разность двух нормально распределенных с.в. тоже нормально распределена со средним coeff_sum0 - coeff_sum1 и дисперсией g0^2 + g1^2 и посчитаем вероятность, что нормально распределенная случайная величина больше 0 с помощью библиотеки scipy 

In [72]:
def prob_policy(coef_sum_0, coef_sum_1, g0, g1) -> float:
    return scipy.stats.norm.sf(0, coef_sum_0 - coef_sum_1, g0**2 + g1**2)

In [73]:
pi_0 = prob_policy(test_data['coeff_sum0'], test_data['coeff_sum1'], test_data['g0'], test_data['g1'])

$\pi_1$

In [100]:
y_pred_0 = scipy.special.logit(model.predict_proba(test_X_0)[:, 1])
y_pred_1 = scipy.special.logit(model.predict_proba(test_X_1)[:, 1])

In [101]:
pi_1 = prob_policy(y_pred_0, y_pred_1, test_data['g0'], test_data['g1'])

посчитаем Clipped IPS

In [102]:
def get_cips(p0, p1, y, l: float = 10):
    return (np.minimum(np.nan_to_num(p1 / p0), l) * y).mean()

In [103]:
get_cips(pi_0, pi_1, test_data['clicks'].to_numpy())

0.05240209838883261